In [1]:
import torch as pt, numpy as np
from astrotools import skymap
from sklearn.preprocessing import OneHotEncoder

print(f"PyTorch version: {pt.__version__}")

PyTorch version: 1.9.0


In [2]:
def train_test_split():
    file = np.load("data/cosmic_ray_sphere/dataset_HAP.npz")
    x_train, x_test = file["data"][:-10000], file["data"][-10000:]
    labels = file["label"]
    y_train, y_test = labels[:-10000], labels[-10000:]
    return x_train, x_test, y_train, y_test

In [3]:
X_train, X_test, y_train, y_test = train_test_split()
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((40000, 500, 4), (10000, 500, 4), (40000,), (10000,))

In [4]:
test_id = 0
example_map = X_test[test_id]
skymap.eventmap(example_map[:,0:3].T, c=example_map[:,3], cblabel='Energy (normed)', opath="skymap_%i.png" % test_id)

(<Figure size 864x432 with 0 Axes>, <HammerAxes:>)

<Figure size 864x432 with 0 Axes>

In [5]:
train_points, test_points, train_features, test_features = pt.from_numpy(X_train[...,:3]).type(pt.float32), pt.from_numpy(X_test[...,:3]).type(pt.float32), pt.from_numpy(X_train[...,-1, np.newaxis]).type(pt.float32), pt.from_numpy(X_test[...,-1, np.newaxis]).type(pt.float32)
train_input_data, test_input_data = [train_points, train_features],[test_points, test_features]

In [6]:
from torch.utils.data import TensorDataset
train_set = TensorDataset(train_points,train_features)

In [7]:
LAYERS = pt.nn
F = pt.nn.functional
NODES = 16

def kernel_NN(data, nodes=NODES):
    x1, x2 = data
    delta = x1 - x2
    x = pt.cat((x1,delta), axis=-1)
    input_shape = x.shape[2]

    x = LAYERS.Linear(input_shape,nodes)(x)
    x = F.relu(x)
    x = LAYERS.BatchNorm1d(nodes)(x)

    x = LAYERS.Linear(nodes,nodes)(x)
    x = F.relu(x)
    x = LAYERS.BatchNorm1d(nodes)(x)

    x = LAYERS.Linear(nodes,nodes)(x)
    x = F.relu(x)
    x = LAYERS.BatchNorm1d(nodes)(x)
    return x

In [9]:
class DGCNN(LAYERS.Module):
    def __init__(self):
        super(DGCNN, self).__init__()

        self.HL1 = EdgeConv(lambda a : kernel_NN(a, nodes=8), next_neighbors = 5)
        self.HL2 = EdgeConv(lambda a : kernel_NN(a, nodes=16), next_neighbors = 8)
        self.HL3 = EdgeConv(lambda a : kernel_NN(a, nodes=32), next_neighbors = 16)
        # self.out = LAYERS.Linear(,2)
    
    def forward(self,train_arr):
        points_in, feats_in = train_arr

        l1 = self.HL1([points_in, feats_in])
        l1 = F.relu(l1)

        l2 = self.HL2([points_in, l1])
        l2 = F.relu(l2)

        l3 = self.HL1([points_in, l2])
        l3 = F.relu(l3)
        # l3 = GlobalAveragePooling('channels_first')(l3)

        # out = self.out(l3)
        out = LAYERS.Softmax(dim=1)(out)
        return out
        

In [10]:
model = DGCNN()

In [11]:
model(train_input_data[:64])

NotImplementedError: 